In [3]:
codon_chart = {'AUG': 'Met', 'UAA': 'STOP', 'UAG': 'STOP', 'UGA': 'STOP', 
               'UUU': 'Phe', 'UUC': 'Phe', 'UUA': 'Leu', 
               'UCU': 'Ser', 'UCC': 'Ser', 'UCA': 'Ser', 'UCG': 'Ser',
              'UAU': 'Tyr', 'UAC': 'Tyr', 'UGU': 'Cys', 'UGC': 'Cys', 'UGG': 'Trp',
              'CUU': 'Leu', 'CUC': 'Leu', 'CUA': 'Leu', 'CUG': 'Leu',
              'CCU': 'Pro', 'CCC': 'Pro', 'CCA': 'Pro', 'CCG': 'Pro',
              'CAU': 'His', 'CAC': 'His', 'CAA': 'Gln', 'CAG': 'Gln',
              'CGU': 'Arg', 'CGC': 'Arg', 'CGA': 'Arg', 'CGG': 'Arg',
              'AAU': 'Ile', 'AUC': 'Ile', 'AUA': 'Ile',
              'ACU': 'Thr', 'ACC': 'Thr', 'ACA': 'Thr', 'ACG': 'Thr',
              'AAU': 'Asn', 'AAC': 'Asn', 'AAA': 'Lys', 'AAG': 'Lys',
              'AGU': 'Ser', 'AGC': 'Ser', 'AGA': 'Arg', 'AGG': 'Arg',
              'GUU': 'Val', 'GUC': 'Val', 'GUA': 'Val', 'GUG': 'Val',
              'GCU': 'Ala', 'GCC': 'Ala', 'GCA': 'Ala', 'GCG': 'Ala',
              'GAU': 'Asp', 'GAC': 'Asp', 'GAA': 'Glu', 'GAG': 'Glu',
              'GGU': 'Gly', 'GGC': 'Gly', 'GGA': 'Gly', 'GGG': 'Gly'}

dna_seq = input('Please type your DNA sequence:')
def dnaseq_aminoacidseq(dna_seq):
    rna_seq = dna_seq.replace('T', 'U') #Convert to mRNA seq
    amino_acids = []
    for i in range(0, len(rna_seq), 3): # Range from 1st to last index, in steps of 3 (bc every 3 nucleotides is 1 codon)
        codon = rna_seq[i:i+3]
        if codon in codon_chart:
            amino_acid = codon_chart[codon]
            if amino_acid == 'STOP':
                break
            amino_acids.append(amino_acid)
    amino_acids_string = ''.join([str(element) for element in amino_acids]) # Converting to string
    return amino_acids_string

print(dnaseq_aminoacidseq(dna_seq))

Please type your DNA sequence: ATGCGAGGA


MetArgGly


In [ ]:
from Bio.Blast import NCBIWWW, NCBIXML

query_sequence = "MALWMRLLPLLALLALWGPDPAAA" 

result_handle = NCBIWWW.qblast("blastp", "pdb", query_sequence)

blast_records = NCBIXML.parse(result_handle)

for blast_record in blast_records:
    print(f"Query: {blast_record.query}")
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            print(f"Hit: {alignment.title}")
            print(f"Score: {hsp.score}, E-value: {hsp.expect}")
